# SlideRule API: Interactive Widget

SlideRule is an on-demand science data processing service that runs in the cloud and responds to REST API calls to process and return science results.

SlideRule can be accessed by any http client (e.g. curl) by making GET and POST requests to the SlideRule service. For the purposes of this document, all requests to SlideRule will originate from a Python script using Python's requests module.

This notebook uses [Jupyter widgets](https://ipywidgets.readthedocs.io) to set parameters for using the SlideRule API.
Regions of interest for submitting to SlideRule are drawn on a [leaflet](https://ipyleaflet.readthedocs.io) map.  Multiple polygons can be submitted at a given time.

The results from SlideRule can be displayed on the interactive [leaflet](https://ipyleaflet.readthedocs.io) map along with additional contextual layers.

#### Load necessary packages

In [ ]:
from sliderule import icesat2, ipysliderule, io
import ipywidgets as widgets
import logging
import warnings
# autoreload
%load_ext autoreload
%autoreload 2
# turn off warnings for demo
warnings.filterwarnings('ignore')

#### Set options for making science data processing requests to SlideRule

In [ ]:
# set the url for the sliderule service
# set the logging level
icesat2.init("icesat2sliderule.org", loglevel=logging.WARNING)

In [ ]:
# display widgets for setting SlideRule parameters
SRwidgets = ipysliderule.widgets()
widgets.VBox([
    SRwidgets.asset,
    SRwidgets.release,
    SRwidgets.surface_type,
    SRwidgets.length,
    SRwidgets.step,
    SRwidgets.confidence,
    SRwidgets.land_class,
    SRwidgets.iteration,
    SRwidgets.spread,
    SRwidgets.count,
    SRwidgets.window,
    SRwidgets.sigma,
    SRwidgets.projection
])

#### Select regions of interest for submitting to SlideRule

There are 3 projections available within SlideRule for mapping
- [Global (Web Mercator)](https://epsg.io/3857)
- [North (Alaska Polar Stereographic)](https://epsg.io/5936)
- [South (Antarctic Polar Stereographic)](https://epsg.io/3031)

In [ ]:
# create ipyleaflet map in specified projection
m = ipysliderule.leaflet(SRwidgets.projection.value)
m.map

#### Build and transmit requests to SlideRule

In [ ]:
%%time
# sliderule asset and data release
asset = SRwidgets.asset.value
release = SRwidgets.release.value

# build sliderule parameters using latest values from widget
parms = {
    # surface type: 0-land, 1-ocean, 2-sea ice, 3-land ice, 4-inland water
    "srt": SRwidgets.surface_type.index,
    # length of ATL06-SR segment in meters
    "len": SRwidgets.length.value,
    # step distance for successive ATL06-SR segments in meters
    "res": SRwidgets.step.value,
    # confidence level for PE selection
    "cnf": SRwidgets.confidence.value,
    # ATL08 land surface classifications
    "atl08_class": list(SRwidgets.land_class.value),
    # maximum iterations, not including initial least-squares-fit selection
    "maxi": SRwidgets.iteration.value,
    # minimum along track spread
    "ats": SRwidgets.spread.value,
    # minimum PE count
    "cnt": SRwidgets.count.value,
    # minimum height of PE window in meters
    "H_min_win": SRwidgets.window.value,
    # maximum robust dispersion in meters
    "sigma_r_max": SRwidgets.sigma.value
}

# create an empty geodataframe
gdf = icesat2.__emptyframe()

# for each region of interest
for poly in m.regions:
    # add polygon from map to sliderule parameters
    parms["poly"] = poly 
    # make the request to the SlideRule (ATL06-SR) endpoint
    # and pass it the request parameters to request ATL06 Data
    gdf = gdf.append(icesat2.atl06p(parms, asset, version=release))

#### Review GeoDataFrame output
See http://icesat2sliderule.org/rtd/user_guide/ICESat-2.html#elevations for descriptions of each column

In [ ]:
print(f'Returned {gdf.shape[0]} records')
gdf.head()

#### Add GeoDataFrame to map

Will limit the plot to be up to 10000 points from the GeoDataFrame as a default.

There are also contextual layers available for each projection.

Global (Web Mercator)
- [USGS 3DEP Hillshade](https://apps.nationalmap.gov/3depdem/)
- [ESRI Imagery](https://www.arcgis.com/home/item.html?id=10df2279f9684e4a9f6a7f08febac2a9)
- [Randolph Glacier Inventory (RGI)](http://glims.colorado.edu/glacierdata/)

North (Alaska Polar Stereographic)
- [ESRI Imagery](http://goto.arcgisonline.com/maps/Arctic_Imagery)
- [ArcticDEM](https://www.pgc.umn.edu/data/arcticdem)
 
South (Antarctic Polar Stereographic)
- [Landsat Image Mosaic of Antarctica (LIMA)](https://lima.usgs.gov/)
- [MODIS Mosaic of Antarctica (MOA)](https://nsidc.org/data/nsidc-0280)
- [Radarsat Antarctic Mapping Project (RAMP)](https://nsidc.org/data/NSIDC-0103)

In addition, most [xyzservice providers](https://xyzservices.readthedocs.io/en/stable/introduction.html) can be added as contextual layers to Web Mercator maps

In [ ]:
widgets.VBox([
    SRwidgets.variable,
    SRwidgets.cmap,
    SRwidgets.reverse,
    SRwidgets.layers
])

In [ ]:
m.add_layer(layers=SRwidgets.layers.value)
m.GeoData(gdf, column_name=SRwidgets.variable.value, cmap=SRwidgets.colormap)

#### Remove contextual layers from map

In [ ]:
m.remove_layer(layers=m.layers)

#### Save GeoDataFrame to output file
- [pytables HDF5](https://www.pytables.org/): easily read back as a Geopandas GeoDataFrame
- [netCDF](https://www.unidata.ucar.edu/software/netcdf): interoperable with other programs

In [ ]:
display(SRwidgets.filesaver)

In [ ]:
# append sliderule api version to attributes
version = icesat2.get_version()
parms['version'] = version['icesat2']['version']
parms['commit'] = version['icesat2']['commit']
# save to file in format (HDF5 or netCDF)
io.to_file(gdf, SRwidgets.file,
    format=SRwidgets.format,
    driver='pytables',
    parameters=parms,
    regions=m.regions,
    verbose=True)

#### Read GeoDataFrame from input file
- [pytables HDF5](https://www.pytables.org/)
- [netCDF](https://www.unidata.ucar.edu/software/netcdf)

In [ ]:
display(SRwidgets.fileloader)

In [ ]:
# read from file in format (HDF5 or netCDF)
gdf = io.from_file(SRwidgets.file,
    format=SRwidgets.format,
    driver='pytables')

#### Review GeoDataFrame input from file

In [ ]:
gdf.head()